In [ ]:
!pip install -q espnet torch editdistance


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.27.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from espnet2.bin.asr_train import main
import yaml, os, re, torch
import sentencepiece as spm
import pandas as pd
import librosa
import soundfile as sf
def load_and_resample_audio(wav_path):
    y, sr = librosa.load(wav_path, sr=48000)
    y_16k = librosa.resample(y, orig_sr=48000, target_sr=16000)
    return y_16k, 16000


Failed to import Flash Attention, using ESPnet default: No module named 'flash_attn'


/usr/local/lib/python3.10/dist-packages/typeguard/__init__.py:1016: UserWarning: no type annotations present -- not typechecking espnet2.asr.frontend.fused.FusedFrontends.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [ ]:
!git clone https://github.com/espnet/espnet.git
%cd espnet
!pip install -e .


fatal: destination path 'espnet' already exists and is not an empty directory.
/content/espnet
Obtaining file:///content/espnet
  Preparing metadata (setup.py) ... done
  Attempting uninstall: espnet
    Found existing installation: espnet 202409
    Uninstalling espnet-202409:
      Successfully uninstalled espnet-202409
  Running setup.py develop for espnet


In [ ]:
#extracting Marathi dataset
os.makedirs("marathi_data", exist_ok=True)
!wget -O marathi_data/mr_in_female.zip https://us.openslr.org/resources/64/mr_in_female.zip
!unzip marathi_data/mr_in_female.zip -d marathi_data/
!rm marathi_data/mr_in_female.zip

--2024-11-30 18:23:35--  https://us.openslr.org/resources/64/mr_in_female.zip
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712155683 (679M) [application/zip]
Saving to: ‘marathi_data/mr_in_female.zip’

marathi_data/mr_in_ 100%[===================>] 679.16M  20.1MB/s    in 35s     

2024-11-30 18:24:11 (19.2 MB/s) - ‘marathi_data/mr_in_female.zip’ saved [712155683/712155683]

Archive:  marathi_data/mr_in_female.zip
  inflating: marathi_data/line_index.tsv  
  inflating: marathi_data/mrt_01523_00028548203.wav  
  inflating: marathi_data/mrt_01523_00029882518.wav  
  inflating: marathi_data/mrt_01523_00032624107.wav  
  inflating: marathi_data/mrt_01523_00053998544.wav  
  inflating: marathi_data/mrt_01523_00064455618.wav  
  inflating: marathi_data/mrt_01523_00087985960.wav  
  inflating: marathi_data/mrt_01523_00119401623.wav  
  inflating: 

In [ ]:
#normalizing Mrathi text
def normalize_marathi_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    replacements = {
        '०': '0', '१': '1', '२': '2', '३': '3', '४': '4', '५': '5', '६': '6', '७': '7', '८': '8', '९': '9'
    }   #converting Marathi numbers
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text


In [ ]:
def load_and_resample_audio(wav_path):
    y, sr = librosa.load(wav_path, sr=48000)
    y_16k = librosa.resample(y, orig_sr=48000, target_sr=16000)
    return y_16k, 16000


In [ ]:
#preprocessing datset
df = pd.read_csv('marathi_data/line_index.tsv', sep='\t', header=None, names=['file_id', 'text'])
df['text'] = df['text'].apply(normalize_marathi_text)


In [ ]:
#training SentencePiece
os.makedirs("data/train", exist_ok=True)
with open("data/train/text_for_tokens.txt", "w") as f:
    for text in df['text']:
        f.write(f"{text}\n")

spm.SentencePieceTrainer.train(
    input="data/train/text_for_tokens.txt",
    model_prefix="data/train/marathi_tokenizer",
    vocab_size=1000,
    character_coverage=1.0,
    model_type='unigram'
)

#creating list of tokens
with open("data/train/tokens.txt", "w") as f:
    with open("data/train/marathi_tokenizer.vocab", "r") as vocab:
        for line in vocab:
            token = line.split('\t')[0]
            f.write(f"{token}\n")


In [ ]:
#splitting data into train-test-validation split using seed = 64
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=64)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=64)


In [ ]:
print(f"Train samples: {len(train_df)}")
print(f"Valid samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")


Train samples: 1255
Valid samples: 157
Test samples: 157


In [ ]:
def prepare_data_files(df, split_name):
    os.makedirs(f"data/{split_name}", exist_ok=True)
    os.makedirs(f"data/{split_name}/wav_16k", exist_ok=True)

    with open(f"data/{split_name}/wav.scp", "w") as wav_file, \
         open(f"data/{split_name}/text", "w") as text_file, \
         open(f"data/{split_name}/utt2spk", "w") as utt2spk_file:
        for idx, row in df.iterrows():
            uttid = f"utterance_{idx}"
            orig_wav_path = f"/content/espnet/marathi_data/{row['file_id']}.wav"
            new_wav_path = f"data/{split_name}/wav_16k/{row['file_id']}_16k.wav"

            #resampling audio
            y_16k, sr = load_and_resample_audio(orig_wav_path)
            sf.write(new_wav_path, y_16k, sr)

            wav_file.write(f"{uttid} {new_wav_path}\n")
            text_file.write(f"{uttid} {row['text']}\n")
            utt2spk_file.write(f"{uttid} global\n")

    #creating shape file using audio duration
    with open(f"data/{split_name}/speech.scp.shape", "w") as f:
        for idx, row in df.iterrows():
            new_wav_path = f"data/{split_name}/wav_16k/{row['file_id']}_16k.wav"
            y_16k, _ = librosa.load(new_wav_path, sr=16000)
            shape = len(y_16k)
            f.write(f"utterance_{idx}\t{shape}\n")


In [ ]:
prepare_data_files(train_df, "train")
prepare_data_files(val_df, "valid")
prepare_data_files(test_df, "test")


In [ ]:
config = {
    "optim": "adam",
    "optim_conf": {"lr": 4e-5},
    "max_epoch": 50,
    "batch_size": 16,
    "num_workers": 4,   #4 parallel data loading workers
    "frontend": "default",
    "specaug_conf": {
        "apply_time_warp": True,
        "time_warp_window": 5,
        "time_warp_mode": "bicubic",
        "apply_freq_mask": True,
        "freq_mask_width_range": [0, 27],
        "num_freq_mask": 2,
        "apply_time_mask": True,
        "time_mask_width_range": [0, 100],
        "num_time_mask": 2
    },
    "encoder": "conformer",
    "decoder": "rnn",
        "encoder_conf": {
        "output_size": 256,
        "attention_heads": 4,
        "linear_units": 1024,
        "num_blocks": 12,
        "dropout_rate": 0.1,
        "positional_dropout_rate": 0.1,
        "attention_dropout_rate": 0.1
    },
        "decoder_conf": {
        "rnn_type": "lstm",
        "num_layers": 2,
        "hidden_size": 512,
        "dropout": 0.1,
    },
    "token_type": "bpe",   #changed token_type from "char" to "bpe"
    "bpemodel": "data/train/marathi_tokenizer.model",
    "preprocessor": "default",
    "use_tensorboard": True,
    "patience": 7,    #waits 7 epochs before making changes
    "val_scheduler_criterion": ["valid", "loss", "min"],    #adjusting learning rate based on validation loss
    "early_stopping_criterion": ["valid", "loss", "min", 3],    #stopping if validation loss does not improve
    "best_model_criterion": [["valid", "loss", "min"]]    #saving best model based on validation loss
}

In [ ]:
#saving configuration
os.makedirs("conf", exist_ok=True)

with open("conf/train_asr.yaml", "w") as f:
    yaml.dump(config, f)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir exp/marathi_asr/tensorboard

<IPython.core.display.Javascript object>

In [ ]:
!python -m espnet2.bin.asr_train \
   --config conf/train_asr.yaml \
   --train_data_path_and_name_and_type data/train/wav.scp,speech,sound \
   --train_data_path_and_name_and_type data/train/text,text,text \
   --valid_data_path_and_name_and_type data/valid/wav.scp,speech,sound \
   --valid_data_path_and_name_and_type data/valid/text,text,text \
   --token_list data/train/tokens.txt \
   --train_shape_file data/train/speech.scp.shape \
   --valid_shape_file data/valid/speech.scp.shape \
   --fold_length 160000 \
   --output_dir exp/marathi_asr \
   --ngpu 1



Streaming output truncated to the last 5000 lines.
[56d3b0e3239c] 2024-11-30 18:59:03,570 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
[56d3b0e3239c] 2024-11-30 18:59:03,713 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
[56d3b0e3239c] 2024-11-30 18:59:03,714 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
[56d3b0e3239c] 2024-11-30 18:59:03,880 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to

#Testing

In [ ]:
!python -m espnet2.bin.asr_inference \
    --asr_model_file exp/marathi_asr/valid.loss.best.pth \
    --data_path_and_name_and_type data/test/wav.scp,speech,sound \
    --output_dir exp/marathi_asr/decode_test \
    --allow_variable_data_keys true

Failed to import Flash Attention, using ESPnet default: No module named 'flash_attn'
2024-11-30 19:21:49.456181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 19:21:49.477768: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 19:21:49.484650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 19:21:50.692672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/espnet/espnet2/enh/loss/criterions/time_domain.py:446: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autoc

In [ ]:
import editdistance

def calculate_error_rates(ref_file, hyp_file):
    with open(ref_file) as f1, open(hyp_file) as f2:
        refs = [line.strip().split(' ', 1)[1] for line in f1]
        hyps = [line.strip().split(' ', 1)[1] for line in f2]

        total_wer = 0
        total_cer = 0

        for ref, hyp in zip(refs, hyps):

            #calculating WER
            ref_words = ref.split()
            hyp_words = hyp.split()
            word_edits = editdistance.eval(ref_words, hyp_words)
            wer = word_edits / len(ref_words) if ref_words else 1
            total_wer += wer

            #calculating CER
            char_edits = editdistance.eval(ref, hyp)
            cer = char_edits / len(ref) if ref else 1
            total_cer += cer

        avg_wer = (total_wer / len(refs)) * 100
        avg_cer = (total_cer / len(refs)) * 100

        return avg_wer, avg_cer

wer, cer = calculate_error_rates(
    'data/test/text',
    'exp/marathi_asr/decode_test/1best_recog/text'
)
print(f"WER: {wer:.2f}%")
print(f"CER: {cer:.2f}%")

WER: 67.99%
CER: 51.49%


#Testing
##Common Voice 11 Marathi

In [ ]:
from datasets import load_dataset
from pathlib import Path


RESAMPLED_AUDIO_DIR = "resampled_audio"
os.makedirs(RESAMPLED_AUDIO_DIR, exist_ok=True)

def resample_audio(input_path, output_path, target_sr=16000):
    audio, sr = librosa.load(input_path, sr=None)
    if sr != target_sr:
        audio = librosa.resample(y=audio, orig_sr=sr, target_sr=target_sr)
    sf.write(output_path, audio, target_sr)
    return output_path

#loading Common Voice 11
cv_train = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="train")
cv_test = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test")
cv_valid = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="validation")


combined_data = {
    'path': [],
    'sentence': []
}

for dataset in [cv_train, cv_test, cv_valid]:
    for item in dataset:
        original_path = item['audio']['path']
        filename = Path(original_path).name
        resampled_path = os.path.join(RESAMPLED_AUDIO_DIR, filename)

        if not os.path.exists(resampled_path):
            resampled_path = resample_audio(original_path, resampled_path)

        combined_data['path'].append(resampled_path)
        combined_data['sentence'].append(item['sentence'])

#splitting data into training-testing-validation splits with seed = 64
train_paths, temp_paths, train_sentences, temp_sentences = train_test_split(
    combined_data['path'], combined_data['sentence'],
    test_size=0.2, random_state=64
)

valid_paths, test_paths, valid_sentences, test_sentences = train_test_split(
    temp_paths, temp_sentences,
    test_size=0.5, random_state=64
)

print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(valid_paths)}")
print(f"Test samples: {len(test_paths)}")



Train samples: 4594
Validation samples: 574
Test samples: 575


In [ ]:
os.makedirs("cv_data/cv_test", exist_ok=True)
with open("cv_data/cv_test/wav.scp", "w") as wav_file, \
     open("cv_data/cv_test/text", "w") as text_file, \
     open("cv_data/cv_test/utt2spk", "w") as utt2spk_file:

    for idx, (path, text) in enumerate(zip(test_paths, test_sentences)):
        uttid = f"utterance_{idx}"
        wav_file.write(f"{uttid} {path}\n")
        text_file.write(f"{uttid} {text}\n")
        utt2spk_file.write(f"{uttid} global\n")


In [ ]:
!python -m espnet2.bin.asr_inference \
    --asr_model_file exp/marathi_asr/valid.loss.best.pth \
    --data_path_and_name_and_type cv_data/cv_test/wav.scp,speech,sound \
    --output_dir exp/marathi_asr/cv_decode_test \
    --allow_variable_data_keys true

Streaming output truncated to the last 5000 lines.
2024-11-30 20:20:29,383 (asr_inference:509) INFO: speech length: 66816
2024-11-30 20:20:29,442 (beam_search:428) INFO: decoder input length: 130
2024-11-30 20:20:29,442 (beam_search:429) INFO: max output length: 130
2024-11-30 20:20:29,442 (beam_search:430) INFO: min output length: 0
2024-11-30 20:20:29,443 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
2024-11-30 20:20:29,444 (nets_utils:157) WARNING: Using make_pad_mask with a list of lengths is not tracable. If you try to trace this function with type(lengths) == list, please change the type of lengths to torch.LongTensor.
2024-11-30 20:20:30,702 (beam_search:444) INFO: end detected at 18
2024-11-30 20:20:30,702 (beam_search:476) INFO: -47.13 * 0.5 = -23.56 for decoder
2024-11-30 20:20:30,702 (beam_search:476) INFO: -38.56 * 0.5 

In [ ]:
wer, cer = calculate_error_rates(
    'cv_data/cv_test/text',
    'exp/marathi_asr/cv_decode_test/1best_recog/text'
)

print(f"WER: {wer:.2f}%")
print(f"CER: {cer:.2f}%")

with open("cv_test_results.txt", "w") as f:
    f.write(f"WER: {wer:.2f}%\n")
    f.write(f"CER: {cer:.2f}%\n")

WER: 107.68%
CER: 81.22%
